# Sistema de Análise de Sentimento NLTK

## 1. Importando pacotes

In [47]:
# Importações referentes ao NLTK
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

### A classe SentimetIntensityAnalyzer foi otimizada para trabalhar com a língua inglesa, por isso este projeto irá usar o google translator

In [48]:
# Importações referentes ao google translator

# !pip install googletrans==4.0.0-rc1
from googletrans import Translator

## 2. Configurando o NLTK

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')
nltk.download('vader_lexicon')
nltk.download('wordnet')

## 3. Construindo a classe Sentiment

### A classe criada chamada Sentiment tem a função de faciliar os tratamentos necessários para a obtenção dos sentimentos das frases, tendo as funções:

1. import_text [Inserir os textos na classe]
2. get_text [Obter os textos]
3. to_english [Traduzir as frases português para inglês]
4. tokenize [Tokenizador]
5. lemma [Lematizador]
6. sentiment [Obter de fato os sentimentos]

No final de tudo a Classe Sentiment irá retornar um dicionário de: positive, negative e neutral.

In [203]:
class Sentiment:
  def __init__ (self):
    self.text = ''

  # Um processo que trabalha junto com o lemma
  # Este processo tem a função de converter o pos_tag para algo que o lemmatizer entende
  def __LEMMA__(self, treebank_tag):
    if treebank_tag.startswith('J'):
      return wordnet.ADJ
    elif treebank_tag.startswith('V'):
      return wordnet.VERB
    elif treebank_tag.startswith('N'):
      return wordnet.NOUN
    elif treebank_tag.startswith('R'):
      return wordnet.ADV
    else:
      return wordnet.NOUN


  # Entrada do texto
  def import_text(self, text):
    self.text = text


  # Obter o texto
  def get_text(self):
    return self.text


  # Tranformar o texto português em inglês
  def to_english(self, force=False):
    text_ = []
    translator = Translator()
    token = self.tokenize("sent")

    for t in token:
      for x in t:
        en = translator.translate(x, src='pt', dest='en').text
        text_.append(en)

    original = self.text

    if force:
      self.text = text_

    return {"original": original, "english": text_}


  # Tokenizando o texto
  # Este processo é mais usado dentro de outras funções
  # Se for separar em palavras use o _type = "word"
  # Se for separar em sentenças use o _type = "sent"
  def tokenize(self, _type="word"):
    text = []

    if _type == "word":
      for t in self.text:
        w = nltk.word_tokenize(t, language="portuguese")
        text.append(w)

    elif _type == "sent":
      for t in self.text:
        w = nltk.sent_tokenize(t, language="portuguese")
        text.append(w)

    return text


  # Transforma as palavras em sua forma básica
  def lemma(self, force=False):
    text = []
    lemmatizer = WordNetLemmatizer()

    for lem in self.text:
      w = [word for word in nltk.word_tokenize(lem)]
      tagged = [tag for tag in nltk.pos_tag(w)]

      lemmatized_sentence = ' '.join([lemmatizer.lemmatize(word, self.__LEMMA__(tag)) for word, tag in tagged])

      text.append(lemmatized_sentence)

    original = self.text

    if force:
      self.text = text

    return {"original": original, "lemmatized": text}


  # Transforma o texto em um dicionário de positive, negative e neutral
  def sentiment(self):
    text = []
    sia = SentimentIntensityAnalyzer()
    score = {
        'positive': {
            'total': 0,
            'phrases': []
        },
        'negative': {
            'total': 0,
            'phrases': []
        },
        'neutral': {
            'total': 0,
            'phrases': []
        },
        'total': 0
    }

    for t in self.text:

      s = sia.polarity_scores(t)

      if s['compound'] > 0.4:
        score['positive']['total'] += 1
        score['positive']['phrases'].append(t)
      elif s['compound'] < -0.4:
        score['negative']['total'] += 1
        score['negative']['phrases'].append(t)
      else:
        score['neutral']['total'] += 1
        score['neutral']['phrases'].append(t)

      score['total'] += 1

    return score

## 4. Usando a classe para o sentimento

In [204]:
s = Sentiment()

In [205]:
s.import_text([
    'Gostei deste produto, achei ele demais',
    'Não gostei muito deste produto',
    'Achei interessante este produto',
    'Gostei muito do produto XYZ',
    'Pior produto que já vi'
])

In [206]:
trad = s.to_english(force=True)

# texto em português
print("Português")

for i, x in enumerate(trad['original']):
  print(i+1, ". ", x)


# texto em inglês
print("\nInglês")

for i, x in enumerate(trad['english']):
   print(i+1, ". ", x)

Português
1 .  Gostei deste produto, achei ele demais
2 .  Não gostei muito deste produto
3 .  Achei interessante este produto
4 .  Gostei muito do produto XYZ
5 .  Pior produto que já vi

Inglês
1 .  I liked this product, I found it too much
2 .  I didn't like this product very much
3 .  I found this product interesting
4 .  I really liked the product xyz
5 .  Worst product I've ever seen


In [207]:
posts_sent = s.sentiment()

print(posts_sent['positive'])
print(posts_sent['negative'])
print(posts_sent['neutral'])

{'total': 3, 'phrases': ['I liked this product, I found it too much', 'I found this product interesting', 'I really liked the product xyz']}
{'total': 1, 'phrases': ["Worst product I've ever seen"]}
{'total': 1, 'phrases': ["I didn't like this product very much"]}
